In [48]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()
import requests  

In [3]:
RUNPOD_CHATBOT_URL = os.getenv('RUNPOD_CHATBOT_URL')

In [76]:
def get_chatbot_response(client, model_name, messages, temperature =0):
    input_messages = []
    for message in messages:
        input_messages.append({"role" : message["role"], "content":message["content"]})
        
    response = client.chat.completions.create(
        model = model_name,  # Using the existing model_name variable
        messages=input_messages, 
        temperature=temperature,
        top_p=0.8,
        max_tokens=100,
    )
    
    return response.choices[0].message.content


In [78]:


# Initialize the OpenAI Client with your RunPod API Key and Endpoint URL
client = OpenAI(
    api_key="rpa_YJ2IBMXDYYMJUUPNCL5QFEPD2WQKYU76P34XR5211lrda6",
    base_url="https://api.runpod.ai/v2/t5b0bjb31m2ipr/openai/v1",
)

model_name ="meta-llama/Llama-3.1-8B-Instruct"

In [79]:
messages = [
  {
    "role": "system",
    "content": "\n            You are a customer support Bot for a coffee shop called \"Merry's way\"\n\n            here is the menu for this coffee shop.\n\n            Cappuccino - $4.50\n            Jumbo Savory Scone - $3.25\n            Latte - $4.75\n            Chocolate Chip Biscotti - $2.50\n            Espresso shot - $2.00\n            Hazelnut Biscotti - $2.75\n            Chocolate Croissant - $3.75\n            Dark chocolate (Drinking Chocolate) - $5.00\n            Cranberry Scone - $3.50\n            Croissant - $3.25\n            Almond Croissant - $4.00\n            Ginger Biscotti - $2.50\n            Oatmeal Scone - $3.25\n            Ginger Scone - $3.50\n            Chocolate syrup - $1.50\n            Hazelnut syrup - $1.50\n            Carmel syrup - $1.50\n            Sugar Free Vanilla syrup - $1.50\n            Dark chocolate (Packaged Chocolate) - $3.00\n\n            Things to NOT DO:\n            * DON'T ask how to pay by cash or Card.\n            * Don't tell the user to go to the counter\n            * Don't tell the user to go to place to get the order\n\n\n            You're task is as follows:\n            1. Take the User's Order\n            2. Validate that all their items are in the menu\n            3. if an item is not in the menu let the user and repeat back the remaining valid order\n            4. Ask them if they need anything else.\n            5. If they do then repeat starting from step 3\n            6. If they don't want anything else. Using the \"order\" object that is in the output. Make sure to hit all three points\n                1. list down all the items and their prices\n                2. calculate the total. \n                3. Thank the user for the order and close the conversation with no more questions\n\n            The user message will contain a section called memory. This section will contain the following:\n            \"order\"\n            \"step number\"\n            please utilize this information to determine the next step in the process.\n            \n            produce the following output without any additions, not a single letter outside of the structure below.\n            Your output should be in a structured json format like so. each key is a string and each value is a string. Make sure to follow the format exactly:\n            {\n            \"chain of thought\": \"Write down your critical thinking about what is the maximum task number the user is on right now. Then write down your critical thinking about the user input and its relation to the coffee shop process. Then write down your thinking about how you should respond in the response parameter taking into consideration the Things to NOT DO section. and Focus on the things that you should not do. \",\n            \"step number\": \"Determine which task you are on based on the conversation.\",\n            \"order\": \"this is going to be a list of jsons like so. [{\"item\":put the item name, \"quantity\":put the number that the user wants from this item, \"price\":put the total price of the item }]\n            \"response\": \"write the response to the user\"\n            }\n        "
  },
  {
    "role": "user",
    "content": " \n I want to order one latte please"
  }
]



In [81]:
response = get_chatbot_response(client, model_name, messages)
response

'{\n  "chain of thought": "The user has initiated the conversation by placing an order for a latte. This is the first step in the process. Based on the Things to NOT DO section, I should not ask how to pay or tell the user to go to the counter. I should focus on validating the order and asking if the user needs anything else.",\n  "step number": "1",\n  "order": "[{\\"item\\":\\"Latte\\",\\"quantity\\":1,\\"price\\":\\"$'

In [47]:
response = requests.post(RUNPOD_CHATBOT_URL, headers=headers, json=payload)

In [48]:
response

<Response [401]>

# Prompt Engineering

# Structred Output

In [10]:
system_prompt = """
You are a helpful assitant that answers questions about capitals of countries

Your output should be in a structured json exactly like below, You are not allowed to to write anything other than the json object:
[
    {
        "country": the country that you will get the capital of
        "capital": the capital of the country stated
    }
]

"""

messages =[{"role":"system", "content":system_prompt}]
messages.append({"role":"user", "content":"what is the capital of italy?"})
response = get_chatbot_response(client, model_name, messages)

print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    }
]


In [11]:
type(response)

str

In [13]:
import json
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

In [18]:
type(json_response[0]), json_response[0]['capital']

(dict, 'Rome')

## Input Srtucturing

In [19]:
user_input = """"
Get me the capitals of the following countries:
```
1. Italy
2. Germany
3.France
```
"""

messages = [{"role":"system", "content":system_prompt}]
messages.append({"role":"user", "content":user_input})

response = get_chatbot_response(client, model_name, messages)

print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [20]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

## Give the model time to think (Chain of thought)

# RAG - Retrival Augmented Generation

In [ ]:
user_prompt = """ 
    What's new in iphone 16?
"""

messages = [{"role":'user', "content":user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

# Automatically Extract context data from database

In [2]:
from openai import OpenAI

In [64]:
embedding_client = OpenAI(
    api_key  = "rpa_YJ2IBMXDYYMJUUPNCL5QFEPD2WQKYU76P34XR5211lrda6",
    base_url="https://api.runpod.ai/v2/tyv6vmjx17v5h6/openai/v1",
)

In [65]:
user_prompt = "What's new in iphone 16?"
model_name = os.getenv("MODEL_NAME")
model_name_embedding = "BAAI/bge-small-en-v1.5"

In [67]:
output = embedding_client.embeddings.create(
    input=user_prompt,
    model=model_name_embedding
)


In [69]:


output.data[0].embedding

[-0.05110831558704376,
 -0.03489653766155243,
 0.06350376456975937,
 -0.008884420618414879,
 -0.03724740073084831,
 -0.042040713131427765,
 0.015921736136078835,
 0.031126001849770546,
 -0.01283814013004303,
 0.04246814176440239,
 0.05556579306721687,
 0.01331899780780077,
 -0.07211340963840485,
 0.0014378030318766832,
 0.09635474532842636,
 0.028714081272482872,
 0.11510057002305984,
 -0.14752411842346191,
 -0.07907439768314362,
 0.016150716692209244,
 -0.0367589071393013,
 -0.014563122764229774,
 -0.03975091129541397,
 -0.030057430267333984,
 0.030118491500616074,
 0.06139714643359184,
 0.0010905168019235134,
 -0.012731282971799374,
 -0.019188515841960907,
 -0.1384870558977127,
 -0.00042075058445334435,
 -0.006739642005413771,
 0.0166850034147501,
 0.026943303644657135,
 0.000493737927172333,
 -0.03840756416320801,
 -0.0031103105284273624,
 0.03489653766155243,
 -0.009601891040802002,
 0.06374800950288773,
 0.029599469155073166,
 0.09122559428215027,
 -0.10142283886671066,
 -0.024210

In [21]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [70]:
user_prompt_embeddings = get_embedding(embedding_client,model_name,user_prompt)

In [71]:
user_prompt_embeddings

[[-0.05110831558704376,
  -0.03489653766155243,
  0.06350376456975937,
  -0.008884420618414879,
  -0.03724740073084831,
  -0.042040713131427765,
  0.015921736136078835,
  0.031126001849770546,
  -0.01283814013004303,
  0.04246814176440239,
  0.05556579306721687,
  0.01331899780780077,
  -0.07211340963840485,
  0.0014378030318766832,
  0.09635474532842636,
  0.028714081272482872,
  0.11510057002305984,
  -0.14752411842346191,
  -0.07907439768314362,
  0.016150716692209244,
  -0.0367589071393013,
  -0.014563122764229774,
  -0.03975091129541397,
  -0.030057430267333984,
  0.030118491500616074,
  0.06139714643359184,
  0.0010905168019235134,
  -0.012731282971799374,
  -0.019188515841960907,
  -0.1384870558977127,
  -0.00042075058445334435,
  -0.006739642005413771,
  0.0166850034147501,
  0.026943303644657135,
  0.000493737927172333,
  -0.03840756416320801,
  -0.0031103105284273624,
  0.03489653766155243,
  -0.009601891040802002,
  0.06374800950288773,
  0.029599469155073166,
  0.0912255942